In [1]:
%reset
#%qtconsole
%load_ext autoreload
%autoreload 2
%matplotlib inline 
import matplotlib.pyplot as plt
import functions as fun

import numpy as np
import matplotlib as mpl
from scipy import stats

from scipy.interpolate import interp1d,interp2d
from scipy.optimize import least_squares

import pandas as pd





Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


In [2]:
#for idx_cases in [0,1]:
for idx_cases in [1]:
    if idx_cases==0:
        #Mice separated 1.1936049813018852
        best_param=np.array([ 9.97,  8.4 , -0.48, -1.7 , -0.1 , -3.88])
        best_inputs=np.array([0.86, 0.89, 0.91, 0.94, 0.95, 1.02, 1.14, 0.62])

    if idx_cases==1:
        #Monkeys separated 4.827433845320423
        best_param=np.array([ 4.58,  2.73, -0.09, -2.14,  0.82, -3.35])
        best_inputs=np.array([1.16, 1.48, 1.57, 1.63, 1.65, 1.68, 0.4 ])
        


    g_E=best_param[0]; g_I=best_param[1];beta=10**best_param[2]; # parameters characterizing the relative strength of synaptic connections
    K=500; # average number of E connections per neuron (I connections are 1/4 of that)
    sigma_Lambda_over_Lambda=10**best_param[4]; # ratio between mean and standard deviation of the opsin expression
    CV_K=10**best_param[3]
    J=10**best_param[5]; # Synaptic efficacy in Volts
    L=[0., 10**best_inputs[-1]]; #  Laser intensity that fits the data
    r_X=10**best_inputs[0:-1]


    print(beta)
    #print(CV_K,J*10**3,g_E,g_I,beta)
    #print(r_X[0],r_X[-1])
    #print(L[1])
    #print(sigma_Lambda_over_Lambda)




0.8128305161640993


In [33]:
sigma_t=0.01; # Input noise, determine how smooth the single neuron transfer function is 
tau_rp=0.002; # Refractory period

g_E=best_param[0]; g_I=best_param[1];beta=10**best_param[2]; # parameters characterizing the relative strength of synaptic connections
K=500; # average number of E connections per neuron (I connections are 1/4 of that)
sigma_Lambda_over_Lambda=10**best_param[4]; # ratio between mean and standard deviation of the opsin expression
CV_K=10**best_param[3]
J=10**best_param[5]; # Synaptic efficacy in Volts
L=[0., 10**best_inputs[-1]]; #  Laser intensity that fits the data
#r_X=[10**best_inputs[-2]]
r_X=10**best_inputs[0:-1]
#[20,20,20]; # To speed up the code, I show results onlye for low anf high contrast and laser

Tmax_over_tau_E=200;

p=0.1; # probability of connection between cells. 
sigma_X_over_r_X=0.2; # Presynaptic population has mean rate r_X, Gaussianly distributed with std=0.2* r_X
Lambda=1*10**-3; # mean opsin expression in Volt

sigma_Lambda=sigma_Lambda_over_Lambda*Lambda

# In what follows, I compute W_{AB} starting from the parameters defined above
G_E,G_I=1.0,2.0 # Gain of Excitatory and inhibitory cells and I cells
w_EE=1;w_IE=w_EE/beta;
w_EI=g_E*w_EE;w_II=g_I*w_IE;
w_EX,w_IX=(G_I*fun.gamma*g_E-G_E)*w_EE,(G_I*fun.gamma*g_I-G_E)*w_IE; 
w_X=np.asarray([w_EX,w_IX]);
w=np.zeros((2,2));
w[0,:]=w_EE,-w_EI
w[1,:]=w_IE,-w_II



In [34]:
#'''
mu_tab_max=10.0;
mu_tab=np.linspace(-mu_tab_max,mu_tab_max,200000)
mu_tab=np.concatenate(([-10000],mu_tab))
mu_tab=np.concatenate((mu_tab,[10000]))

phi_tab_E,phi_tab_I=mu_tab*0,mu_tab*0;
for idx in range(len(phi_tab_E)):
    phi_tab_E[idx]=fun.comp_phi_tab(mu_tab[idx],fun.tau_E,tau_rp,sigma_t)
    phi_tab_I[idx]=fun.comp_phi_tab(mu_tab[idx],fun.tau_I,tau_rp,sigma_t)

phi_int_E=interp1d(mu_tab, phi_tab_E, kind='linear')  
phi_int_I=interp1d(mu_tab, phi_tab_I, kind='linear')
#'''

/Users/ale/Research/Projects/Laser_on_E_cells/Single_cell_comparison/Simulate_monkeys/functions.py:53: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  adelleh=integrate.quad(lambda u: f(u),minimo,massimo)
/Users/ale/Research/Projects/Laser_on_E_cells/Single_cell_comparison/Simulate_monkeys/functions.py:53: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  adelleh=integrate.quad(lambda u: f(u),minimo,massimo)


In [35]:
# Generate quenched disorder
M,mu_X_over_r_X_tau,Lambda_i,N_E,N_I=fun.Generate_quenched_disorder(CV_K,J,K,w,w_X,p,Lambda,sigma_Lambda)


In [36]:
print('Computing and saving network response')
T=np.arange(0,Tmax_over_tau_E*fun.tau_E,fun.tau_I/3);

mask_time=T>(10*fun.tau_E)
RATES=-1*np.ones((len(r_X),len(L),N_E+N_I))
DYNA=-1*np.ones((len(r_X),len(L),N_E+N_I,len(T)))
MUS=-1*np.ones((len(r_X),len(L),N_E+N_I))
Lambda_i_L=1./tau_rp*np.ones((len(r_X),len(L),N_E+N_I))
MFT_SOL_R=np.ones((len(r_X),len(L),10))
MFT_SOL_M=np.ones((len(r_X),len(L),10))
Phi=[phi_int_E,phi_int_I];
for idx_r_X in range(len(r_X)):
    for idx_L in range(len(L)):
        r_X_local,L_local=r_X[idx_r_X],L[idx_L];
        print((idx_r_X+1)/len(r_X),(idx_L+1)/len(L))
        DYNA[idx_r_X,idx_L,:,:], MUS[idx_r_X,idx_L,:],Lambda_i_L[idx_r_X,idx_L,:]=fun.High_dimensional_dynamics(T,L_local,r_X_local,M,mu_X_over_r_X_tau,Lambda_i,N_E,N_I,phi_int_E,phi_int_I);
        RATES[idx_r_X,idx_L,:]=np.mean(DYNA[idx_r_X,idx_L,:,mask_time],axis=0)
        print(np.mean(RATES[idx_r_X,idx_L,0:N_E]),
              np.mean(RATES[idx_r_X,idx_L,N_E::]),
              np.std(RATES[idx_r_X,idx_L,0:N_E]),
              np.std(RATES[idx_r_X,idx_L,N_E::]))
    Base_Sim=RATES[idx_r_X,0,:]
    Delta_Sim=RATES[idx_r_X,idx_L,:]-RATES[idx_r_X,0,:]

    moments_of_r_sim=np.zeros(5)
    moments_of_r_sim[0]=np.mean(Base_Sim)
    moments_of_r_sim[1]=np.mean(Delta_Sim)
    moments_of_r_sim[2]=np.std(Base_Sim)
    moments_of_r_sim[3]=np.std(Delta_Sim)
    moments_of_r_sim[4]=np.cov(Base_Sim,Delta_Sim)[0,1]
    print(moments_of_r_sim[:])


Computing and saving network response
0.16666666666666666 0.5
20.87717818517452 32.11327098799928 13.09858769069655 10.491824858609835
0.16666666666666666 1.0
22.25176701460838 34.124022667176426 28.91767501106946 19.69088026280885
[ 23.12439675   1.5018214   13.39679224  27.4934776  -83.15409704]
0.3333333333333333 0.5
41.42372232743912 67.3614763972212 40.4619878547296 37.69863463035562
0.3333333333333333 1.0
42.567769241539224 68.86208470148316 47.53889094267844 42.204764128211636
[  46.61127314    1.21535919   41.2506741    42.85796426 -631.89875612]
0.5 0.5
50.229676150152784 82.38176872216548 40.07829709763999 37.60441236121793
0.5 1.0
51.3326944014441 83.83992687145951 53.786471583225314 48.15790675869964
[  56.66009466    1.17404623   41.63214399   44.36815747 -377.29495103]
0.6666666666666666 0.5
57.09549599636481 94.0991172484517 42.67251816944698 39.615545827251395
0.6666666666666666 1.0
58.169768746594535 95.53482095930735 57.28978873333524 50.989707582793315
[  64.49622025

# Save single cell rate


In [24]:
#np.save('RATES_monkeys',RATES)
np.save('RATES_monkeys_2',RATES)